In [21]:
# Parameters
country_name = "Zimbabwe"


# Optimization Problem Time Series

This notebook optimizes an electrci mix based on : 
- an electricity demand time serie
- a PV production time serie
- a wind production time serie

In [34]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from pulp import *
import sys
from utilities import import_excel, optimize_enr_budget_constraint, format_load_data
import plotly.graph_objects as go
import plotly.express as px
import pickle
from ren_ninja_api import fetch_and_average_data_ren_ninja, get_regular_coordinates
import plotly.io as pio
import os
import glob
import geopandas as gpd
pio.renderers.default='notebook'

Enter country name for file naming: 

In [143]:
year = 2021
# country_name = 'United States'
# state_name = 'AL'
country_name = 'Algeria'
# state_name = state_name
# Directory path
path_input_data = '../input_time_series/'
mode = 'grid'

In [144]:
points_in_world = gpd.read_file('grid_with_centroids_states.geojson')
regions_match = pd.read_excel('match_plexos_iso_codes.xlsx')
regions_match = regions_match[regions_match['Country']==country_name].set_index('region')

In [145]:
# points_in_world

In [146]:
import pandas as pd
from shapely.geometry import Point

# Assuming you already have the `country_codes` DataFrame

# Define the new rows as dictionaries
new_rows = [
    {
        'geometry': Point(108.2772, 14.0583),
        'latitude': 14.0583,
        'index_right': None,
        'pop_est': 97338579.0,
        'continent': 'Asia',
        'name': 'Vietnam',
        'iso_a3': 'VNM',
        'gdp_md_est': 64810
    },
    {
        'geometry': Point(34.8516, 31.0461),
        'latitude': 31.0461,
        'index_right': None,
        'pop_est': 9053300.0,
        'continent': 'Asia',
        'name': 'Israel',
        'iso_a3': 'ISR',
        'gdp_md_est': 38760
    },
    {
    'geometry': Point(2.2137, 46.6034),  # Approximate centroid coordinates of metropolitan France
    'latitude': 46.6034,
    'index_right': None,
    'pop_est': 65273511.0,  # Population estimate for metropolitan France
    'continent': 'Europe',
    'name': 'France',
    'iso_a3': 'FRA',
    'gdp_md_est': 2777530  # GDP estimate for metropolitan France
    },
    {
        'geometry': Point(121.7740, 12.8797),  # Approximate centroid coordinates of the Philippines
        'latitude': 12.8797,
        'index_right': None,
        'pop_est': 109581078.0,  # Population estimate for the Philippines
        'continent': 'Asia',
        'name': 'Philippines',
        'iso_a3': 'PHL',
        'gdp_md_est': 402601  # GDP estimate for the Philippines
    },
     {
        'geometry': Point(101.9758, 4.2105),  # Approximate centroid coordinates of Malaysia
        'latitude': 4.2105,
        'index_right': None,
        'pop_est': 32365999.0,  # Population estimate for Malaysia
        'continent': 'Asia',
        'name': 'Malaysia',
        'iso_a3': 'MYS',
        'gdp_md_est': 358580  # GDP estimate for Malaysia
    }

]

# Convert the list of dictionaries to a DataFrame
new_rows_df = pd.DataFrame(new_rows)

# Concatenate the new DataFrame with the existing DataFrame
points_in_world = pd.concat([points_in_world, new_rows_df], ignore_index=True)


### Get population data for the country of interest

In [147]:
pop_data = pd.read_excel('../population_data_UN.xlsx', header = 1)
pop_data = pop_data.drop('Notes', axis = 1)
pop_data = pop_data.dropna()
total_pop = pop_data['Total Population, as of 1 January (thousands)'].sum()
pop_data['share of world population'] = pop_data['Total Population, as of 1 January (thousands)']/total_pop

In [148]:
pop_data

,"Region, subregion, country or area *",Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,Year,"Total Population, as of 1 January (thousands)",share of world population
0,Burundi,108,BDI,BI,108,Country/Area,910,2021,12386.556,1.573124e-03
1,Comoros,174,COM,KM,174,Country/Area,910,2021,814.006,1.033808e-04
2,Djibouti,262,DJI,DJ,262,Country/Area,910,2021,1097.968,1.394447e-04
3,Eritrea,232,ERI,ER,232,Country/Area,910,2021,3588.083,4.556956e-04
4,Ethiopia,231,ETH,ET,231,Country/Area,910,2021,118743.462,1.508072e-02
...,...,...,...,...,...,...,...,...,...,...
241,Samoa,882,WSM,WS,882,Country/Area,957,2021,216.800,2.753414e-05
242,Tokelau,772,TKL,TK,772,Country/Area,957,2021,1.837,2.333036e-07
243,Tonga,776,TON,TO,776,Country/Area,957,2021,105.635,1.341591e-05
244,Tuvalu,798,TUV,TV,798,Country/Area,957,2021,11.140,1.414808e-06


In [149]:
pop_data['share of world population'] = pop_data['Total Population, as of 1 January (thousands)']/total_pop

In [150]:
share_pop = pop_data[pop_data['Region, subregion, country or area *']==country_name]['share of world population']

In [151]:
pop_data[pop_data['Region, subregion, country or area *']==country_name]

,"Region, subregion, country or area *",Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,Year,"Total Population, as of 1 January (thousands)",share of world population
30,Algeria,12,DZA,DZ,12,Country/Area,912,2021,43812.345,0.005564


In [152]:
pop_data[pop_data['Region, subregion, country or area *']=='Iran']

,"Region, subregion, country or area *",Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,Year,"Total Population, as of 1 January (thousands)",share of world population


In [153]:
global_elec_budget = 14.6e3 #MtCO2eq
country_budget = share_pop*global_elec_budget*10e6/10 #tCO2eq

### Load Time Series

First we load the time series that will be used in the problem. We use the  ```import_excel``` function used in the wavelet decomposition.

#### Demand

In [154]:
def format_load_data(country_name, state_name = None):
    if not os.path.exists(f'../input_time_series/{country_name}/'):
        os.makedirs(f'../input_time_series/{country_name}/')
    if state_name:
        file_path = f'../input_time_series/{country_name}/{country_name}_{state_name}_demand_Plexos_2015.xlsx'
    else: 
        file_path = f'../input_time_series/{country_name}/{country_name}_demand_Plexos_2015.xlsx'

    if os.path.exists(file_path):
        pass
    else: 
        country_codes = pd.read_csv('../countries_codes_and_coordinates.csv' , sep = ',', index_col = 0)
        data = pd.read_csv('../input_time_series/All Demand UTC 2015.csv', index_col =0)
        iso_code = country_codes.loc[country_name,'Alpha-3 code' ].split(' ')[1]
        if state_name:
            print(data.columns)
            column_name = data.columns[data.columns.str.endswith(state_name)].item()
            data[column_name].to_excel(file_path, index =False)
        else:
            column_name = data.columns[data.columns.str.endswith(iso_code)].item()
            data[column_name].to_excel(file_path, index =False)
        
    return file_path.split('/',2)[-1]


In [155]:
# Demand time serie

dpd = 24 # data per day in the time serie
dpy = 365 # data per year :  cut the leap years to 365 years

ndpd = 24 # new data per day for hourly data (for the interpolation)
signal_length = ndpd * dpy
# region_name = regions_match.loc[state_name, 'plexos']
region_name = None
state_name = None
file_name = format_load_data(country_name, region_name)
Load_ts = import_excel(path_input_data,file_name, 
                                    dpd ,ndpd, dpy, 
                                    interp=True, norm = None) # interpolate data from dpd to ndpd numper of points per day

mean_load = pd.read_excel(path_input_data+file_name).mean().iloc[0]

#### Wind production

In [156]:
# Wind time serie 

dpd = 24 # data per day
dpy = 365 # data per year :  cut the leap years to 365 years

# We interpolate so that we have hourly data
ndpd = 24 # new data per day for hourly data (for the interpolation)
signal_length = ndpd * dpy

folder = path_input_data + f'/{country_name}'


if state_name : 
    partie_name_file = f'{mode}_locations_averaged_wind_{country_name}_{state_name}_{year}.xlsx'
else: 
    partie_name_file = f'{mode}_locations_averaged_wind_{country_name}_{year}.xlsx' 
chemin_pattern = os.path.join(folder, f'*{partie_name_file}*')
fichiers_trouves = glob.glob(chemin_pattern)
print(fichiers_trouves)

if len(fichiers_trouves)==0:
    print('collecting data')
    fetch_and_average_data_ren_ninja(country_name, 1, ['wind'], points_in_world, state = state_name, year=year, save = True, coordinates = mode)

fichiers_trouves = glob.glob(chemin_pattern)
file_name = fichiers_trouves[0].split('/',2)[-1]
Wind_ts = import_excel(path_input_data,file_name, 
                                dpd ,ndpd, dpy, 
                                interp=True, norm = None) # interpolate data from dpd to ndpd numper of points per day

mean_wind = pd.read_excel(path_input_data+file_name).mean().iloc[0]

['../input_time_series//Algeria\\ren_ninja_2_grid_locations_averaged_wind_Algeria_2021.xlsx']


#### PV production

In [157]:
# Wind time serie 

dpd = 24 # data per day
dpy = 365 # data per year :  cut the leap years to 365 years

# We interpolate so that we have hourly data
ndpd = 24 # new data per day for hourly data (for the interpolation)
signal_length = ndpd * dpy

folder = path_input_data + f'/{country_name}'


if state_name : 
    partie_name_file = f'{mode}_locations_averaged_pv_{country_name}_{state_name}_{year}.xlsx'
else: 
    partie_name_file = f'{mode}_locations_averaged_pv_{country_name}_{year}.xlsx' 
chemin_pattern = os.path.join(folder, f'*{partie_name_file}*')
fichiers_trouves = glob.glob(chemin_pattern)
print(fichiers_trouves)

if len(fichiers_trouves)==0:
    print('collecting data')
    fetch_and_average_data_ren_ninja(country_name, 1, ['pv'], points_in_world, state = state_name, year=year, save = True, coordinates = mode)

fichiers_trouves = glob.glob(chemin_pattern)
file_name = fichiers_trouves[0].split('/',2)[-1]
PV_ts = import_excel(path_input_data,file_name, 
                                dpd ,ndpd, dpy, 
                                interp=True, norm = None) # interpolate data from dpd to ndpd numper of points per day

mean_pv = pd.read_excel(path_input_data+file_name).mean().iloc[0]

['../input_time_series//Algeria\\ren_ninja_2_grid_locations_averaged_pv_Algeria_2021.xlsx']


### Plot Time Series

In [158]:
colors_dict = {
    'Wind': 'steelblue',        
    'PV': 'gold',
    'Discharge': 'orangered',    
    'SOC': 'darkgreen',           
    'Charge': 'purple',
    'Consumption': 'green',          
    'Dispatchable': 'crimson',       
    'Curtailment': 'cyan'    
}

In [159]:
# Create a Plotly figure
fig = go.Figure()

fig.add_trace(go.Scatter(y=PV_ts, mode='lines', name='PV',marker=dict(color=colors_dict['PV'])))
fig.add_trace(go.Scatter(y=Load_ts, mode='lines', name='Demand',marker=dict(color=colors_dict['Consumption'])))
fig.add_trace(go.Scatter(y=Wind_ts, mode='lines', name='Wind',marker=dict(color=colors_dict['Wind'])))
fig.update_layout(title=f'{country_name} {state_name} 2021', xaxis_title='Day', yaxis_title='Energy (MW))')

# Show the plot
fig.show()

### Description of the problem

#### Equations:

- **Objective function** : 
  - Minimize dispatchable energy: $ \min(\sum{P_{dispatchable}(t)*dt}) $


- **Node Law** : 
  - $(P_{pv}(t) + P_{wind}(t) + P_{dispatchable}(t) - P_{in\_stock}(t) + P_{out\_stock}(t) = P_{demand}(t) + P_{curt}(t))$


- **State of charge**
  - $SOC(t+1)=SOC(t)+P_{in\_stock}(t) - P_{out\_stock}(t)$

#### Contraintes :
- $I_{wind}*P_{wind} + I_{pv}*E_{pv} \leq Budget$
- We want a maximum storage size of 10 hours:  $E_{stock} \leq 10$
- Charging and discharging at the same time is not possible. 


### Implementation
#### Decision Variables:
- `x_pv`: Installed capacity for photovoltaic production.
- `x_wind`: Installed capacity for wind production.
- `ts_dispatchable`: Dispatchable production (can be controlled), time serie.
- `p_ch`: Battery charging power, time serie.
- `p_dech`: Battery discharging power, time serie.
- `SOC_ts`: State of charge of the battery, time serie.
- `p_curt`: Curtailment power (lost energy), time serie.
- `dech_active`: Binary variable indicating if the battery is charging or discharging.

### Run the optimization with GUROBI

**If the optimization has already been run, go to the next part where results can be loaded and analysed in plots.**

In [166]:
optimized_parameters = optimize_enr_budget_constraint(country_name, Load_ts, PV_ts, Wind_ts, 869, 517, country_budget/10, mean_load, save_results = False)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2537910 - for non-commercial use only - registered to ju___@cea.fr
Optimize a model with 35041 rows, 52561 columns and 118420 nonzeros
Model fingerprint: 0x12643624
Variable types: 43802 continuous, 8759 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-04, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+05]
  RHS range        [3e+03, 8e+06]
Found heuristic solution: objective 6.876996e+07
Presolve removed 4 rows and 8764 columns
Presolve time: 0.46s
Presolved: 35037 rows, 43797 columns, 105333 nonzeros
Variable types: 35038 continuous, 8759 integer (8759 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...


Root simplex log...

Ite

### Add results to .csv file with all countries

In [161]:
df_new = pd.DataFrame({"Country" : [country_name],"State":[state_name],"iso_alpha":[optimized_parameters['iso_alpha']],
    "mean_load":[optimized_parameters['mean_consumption']],
    "E_dispatch": [optimized_parameters['E_dispatch']],
    "P_dispatch": [optimized_parameters['dispatchable_capacity']],
    "E_destock": [optimized_parameters['E_destock']],
    "P_pv": [optimized_parameters['pv_capacity']],
    "P_wind": [optimized_parameters['wind_capacity']]
})


In [54]:
all_results_file = "results/optimization_results_world_budget_constraint_country.csv"
file_exists = os.path.isfile(all_results_file)

df_new.to_csv(all_results_file, mode='a', index=False, header=not file_exists)

print(f"Data have been added to {all_results_file}")    

Data have been added to results/optimization_results_world_budget_constraint_country.csv


## Plots

In [167]:
optimized_parameters['pv_capacity']

0.0

In [168]:
#Calcul des émissions liées aux capacités ENR
ghg = optimized_parameters['wind_capacity']*517+optimized_parameters['pv_capacity']*869
print(f'Emissions : {ghg}')
print(f'Initial budget : {country_budget}')

Emissions : 8123845.682028207
Initial budget : 30    8.123846e+07
Name: share of world population, dtype: float64


In [169]:
from plots import plot_ts_optim, plot_pie_energy, plot_storage, plot_stack_production

In [170]:
plot_ts_optim([optimized_parameters['optimized_pv'], optimized_parameters['optimized_wind'], optimized_parameters['optimized_dispatchable'], optimized_parameters['optimized_p_curt'],np.array(Load_ts) ], ['PV', 'Wind', 'Dispatchable', 'Curtailment', 'Consumption'], country_name, colors_dict = colors_dict,savefig=False)

In [15]:
# plot_storage(optimized_parameters['optimized_charge'], optimized_parameters['optimized_discharge'], optimized_parameters['optimized_stock'], country_name, colors_dict = colors_dict, savefig=False)

In [16]:
# E_wind = optimized_parameters['E_wind']
# E_pv = optimized_parameters['E_pv']
# E_dispatch = optimized_parameters['E_dispatch']

In [17]:
# plot_pie_energy([E_wind, E_pv, E_dispatch], country_name, colors_dict =colors_dict, savefig=False)